## How to use pyctcdecode when working with a Huggingface model

In [ ]:
# install huggingface transformers
!pip install transformers==4.5.1

In [ ]:
# load pretrained model
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
# get a single audio file
!wget https://dldata-public.s3.us-east-2.amazonaws.com/1919-142785-0028.wav

In [2]:
# transcribe audio to logits over characters
import soundfile as sf
arr, _ = sf.read('1919-142785-0028.wav')
input_values = asr_processor(arr, return_tensors="pt", sampling_rate=16000).input_values  # Batch size 1
logits = asr_model(input_values).logits.cpu().detach().numpy()[0]

In [6]:
# look at the alphabet of our model defining the labels for the logit matrix we just calculated
print(asr_processor.tokenizer.get_vocab())

{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31}


The vocabulary is in a slightly unconventional shape so we will replace `"<pad>"` with `""` and `"|"` with `" "` as well as the other special tokens (which are essentially unused)

We need to standardize the special tokens and then specifically pass which index is the ctc blank token index (since it's not the last). For that reason we have to manually build the Alphabet and the decoder instead of using the convenience wrapper `build_ctcdecoder`.

In [7]:
from pyctcdecode import Alphabet, BeamSearchDecoderCTC

# make alphabet
vocab_list = list(asr_processor.tokenizer.get_vocab().keys())
# convert ctc blank character representation
vocab_list[0] = ""
# replace special characters
vocab_list[1] = "⁇"
vocab_list[2] = "⁇"
vocab_list[3] = "⁇"
# convert space character representation
vocab_list[4] = " "
# specify ctc blank char index, since conventionally it is the last entry of the logit matrix
alphabet = Alphabet.build_alphabet(vocab_list, ctc_token_idx=0)

# build the decoder and decode the logits
decoder = BeamSearchDecoderCTC(alphabet)
decoder.decode(logits)

'BOIL THEM BEFORE THEY ARE PUT INTO THE SOUP OR OTHER DISH THEY MAY BE INTENDED FOR'